#### Import Libraries

In [12]:
# Import required libraries for execution
import pandas as pd
import datetime
import os
import numpy as np

# Get username of the windows Account to be used in the folder paths
user = os.getenv('username')

#### Define Report Folders and Variables

In [13]:
# Define the folders for input files and final reports

main_folder = "C:/Users/"+user+"/Desktop/Airtel/Source Files/"
report_folder = "C:/Users/"+user+"/Desktop/Airtel/Reports/Lender Report/"

# Dates for file name and invoices to be read

curr_mon = datetime.datetime.today().strftime('%b%y') #'01Feb18'
curr_date = datetime.datetime.today().strftime('%d%b%y') #'01Feb18'
today = datetime.datetime.now().date()
prev_month_last = today.replace(day=1)
lastmonth = prev_month_last - datetime.timedelta(days=1) #datetime.datetime(2018,12,31).date()

# Create monthly folders if not present already

if not os.path.exists(report_folder+"/"+curr_mon):
    os.makedirs(report_folder+"/"+curr_mon)
    
td_file_name = 'TD - 29Jan19'
pd_file_name = 'PD - 29Jan19'

#### Import Data Files

In [14]:
# Excel File with Active/TD/PD details

xls = pd.ExcelFile(main_folder + td_file_name + '.xlsx')

try:
    td_data = pd.read_excel(xls, 'Details')
except:
    td_name = input("Please enter the name of the TD sheet (Airtel changed it again, why am I not suprised?): ")
    td_data = pd.read_excel(xls, td_name)

xls = pd.ExcelFile(main_folder + pd_file_name + '.xlsx')

try:
    pd_details = pd.read_excel(xls, 'PD Base')
except:
    pd_name = input("Please enter the name of the PD sheet (Airtel changed it again, why am I not suprised?): ")
    pd_details = pd.read_excel(xls, pd_name)

try:
    pd_data = pd.read_excel(xls, 'Billing Details')
except:
    pd_name = input("Please enter the name of the PD sheet (Airtel changed it again, why am I not suprised?): ")
    pd_data = pd.read_excel(xls, pd_name)

master_data = pd.read_csv(main_folder + 'master_anuj.csv')
borr_mobile = pd.read_csv(main_folder + 'borr_mobile.csv')
cibil_score = pd.read_csv(main_folder + 'adf_cibil.csv')
phone_models = pd.read_csv(main_folder + 'phone_models.csv')
clix_dob = pd.read_excel(main_folder + 'clix_dob_master.xlsx')
fed_map = pd.read_csv(main_folder + 'Federal ID Mapping.csv')[['Loan App Reference Number','Lender Loan_Ref_no']]

print("Data Reading Complete!")

Please enter the name of the PD sheet (Airtel changed it again, why am I not suprised?): Billing Details
Data Reading Complete!


#### Data Extraction Steps

In [15]:
td_data['td_flag'] = 1
pd_data['pd_flag'] = 1

td_data = td_data.rename(columns={'Loan ID':'Loan App Reference Number','Plan Type':'Type'})

td_flag = td_data[['Loan App Reference Number','Type','td_flag']]

pd_data = pd_data.rename(columns={'Loan ID ':'Loan App Reference Number','PLAN TYPE':'Type'})

pd_flag = pd_data[['Loan App Reference Number','Type','pd_flag']]    

td_pd = td_flag.append(pd_flag).fillna(0).drop_duplicates()

disbursed_data = master_data[master_data['Loan Application Status'] == 'Disbursed']

borr_mobile = borr_mobile[borr_mobile['Loan Application Status'] == 'Disbursed']
borr_mobile_2 = borr_mobile[['Registered Mobile','MSISDN','Loan Reference Number']]
borr_mobile_2 = borr_mobile_2.rename(columns = {'Loan Reference Number':'Loan App Reference Number'})

disbursed_data = pd.merge(disbursed_data, borr_mobile_2, on = 'Loan App Reference Number', how = 'left')
disbursed_data = pd.merge(disbursed_data, td_pd, on = 'Loan App Reference Number', how = 'left')

disbursed_data = disbursed_data[['Loan App Reference Number', 'Registered Mobile', 'MSISDN', 'Loan Application Status',
       'Priority Label', 'Type', 'Borrower: Date of Birth', 'Application Date',
       'Down payment Date', 'Device Delivered Date', 'Approval Date', 'Lender: Account Name',
       'Is Employee', 'pd_flag', 'td_flag']].fillna(0)

msisdn_lender = disbursed_data[['Loan App Reference Number','Lender: Account Name']]

pd_data = pd.merge(pd_data, msisdn_lender, on = 'Loan App Reference Number', how = 'left')
pd_data = pd_data.drop(['pd_flag'], axis=1)
td_data = pd.merge(td_data, msisdn_lender, on = 'Loan App Reference Number', how = 'left')
td_data = td_data.drop(['td_flag'], axis=1)

print("Step 1 completed successfully")

Step 1 completed successfully


In [16]:
cibil_score = cibil_score.rename(columns = {'Raw Score':'CIBIL Score'})

disbursed_data = pd.merge(disbursed_data, cibil_score, on = 'Loan App Reference Number', how = 'left')
disbursed_data = pd.merge(disbursed_data, phone_models, on = 'Loan App Reference Number', how = 'left')
disbursed_data = disbursed_data.fillna('NA')
disbursed_data['missing_flag'] = disbursed_data.apply(lambda x: 1 if (max(x['Is Employee'],x['pd_flag'],x['td_flag'])==0) else 0,axis=1)


pd_data = pd.merge(pd_data, cibil_score, on = 'Loan App Reference Number', how = 'left')
pd_data = pd.merge(pd_data, phone_models, on = 'Loan App Reference Number', how = 'left')
pd_data = pd_data.fillna('NA')
pd_data = pd_data[['Loan App Reference Number','MOBILE_NO', 'REASON_CODE', 'PD_DATE', 'ALTERNATE_CONTACT_NO',
    'BILL_PLAN_NAME', 'BILL_PLAN_ACTIVE_DT', 'BILL_PLAN_INACTIVE_DT',
    'INVOICE_NO', 'INVOICE_RESETS', 'INVOICE_STATEMENT_DATE', 'DUE_DATE',
    'NEW_CHARGES', 'PAID_AMOUNT', 'ADJUSTMENT', 'BALANCE_DUE',
    'LAST_PAY_DATE', 'ACCOUNT_NUMBER', 'Type', 'Lender: Account Name',
    'CIBIL Score', 'Borrower: Account Name',
    'Manufacturer', 'Model']]


td_data = pd.merge(td_data, cibil_score, on = 'Loan App Reference Number', how = 'left')
td_data = pd.merge(td_data, phone_models, on = 'Loan App Reference Number', how = 'left')
td_data = td_data.fillna('NA')
td_data = td_data[['Loan App Reference Number', 'MOBILE_NO', 'Type', 'BILL_PLAN_NAME', 'PLAN_ACTIVE_DT', 'STATUS', 'INVOICE_NO', 'INVOICE_RESETS', 'INVOICE_STATEMENT_DATE', 'DUE_DATE', 'NEW_CHARGES', 'PAID_AMOUNT', 'ADJUSTMENT', 'BALANCE_DUE', 'LAST_PAY_DATE', 'ACCOUNT_NUMBER', 'Lender: Account Name', 'CIBIL Score', 'Borrower: Account Name', 'Manufacturer', 'Model']]

print("Step 2 completed successfully")

Step 2 completed successfully


In [17]:
def date_convert(date):
    try:
        new_date = datetime.datetime.strptime(date,'%d/%m/%Y').date()
    except:
        new_date = date
    return new_date

td_data['INVOICE_STATEMENT_DATE'] = td_data.apply(lambda x: date_convert(x['INVOICE_STATEMENT_DATE']),axis=1)
td_data['LAST_PAY_DATE'] = td_data.apply(lambda x: date_convert(x['LAST_PAY_DATE']),axis=1)
td_data['DUE_DATE'] = td_data.apply(lambda x: date_convert(x['DUE_DATE']),axis=1)

pd_data['PD_DATE'] = pd_data.apply(lambda x: date_convert(x['PD_DATE']),axis=1)
pd_data['BILL_PLAN_ACTIVE_DT'] = pd_data.apply(lambda x: date_convert(x['BILL_PLAN_ACTIVE_DT']),axis=1)
pd_data['BILL_PLAN_INACTIVE_DT'] = pd_data.apply(lambda x: date_convert(x['BILL_PLAN_INACTIVE_DT']),axis=1)
pd_data['INVOICE_STATEMENT_DATE'] = pd_data.apply(lambda x: date_convert(x['INVOICE_STATEMENT_DATE']),axis=1)
pd_data['DUE_DATE'] = pd_data.apply(lambda x: date_convert(x['DUE_DATE']),axis=1)
pd_data['LAST_PAY_DATE'] = pd_data.apply(lambda x: date_convert(x['LAST_PAY_DATE']),axis=1)

disbursed_data['Application Date'] = disbursed_data.apply(lambda x: date_convert(x['Application Date']),axis=1)
disbursed_data['Down payment Date'] = disbursed_data.apply(lambda x: date_convert(x['Down payment Date']),axis=1)
disbursed_data['Device Delivered Date'] = disbursed_data.apply(lambda x: date_convert(x['Device Delivered Date']),axis=1)
disbursed_data['Approval Date'] = disbursed_data.apply(lambda x: date_convert(x['Approval Date']),axis=1)

print("Step 3 completed successfully")

Step 3 completed successfully


In [18]:
writer = pd.ExcelWriter(report_folder+"/"+curr_mon+"/master_disbursed_data_"+curr_date+".xlsx")
disbursed_data.to_excel(writer,'Master Data',index = False)
td_data.to_excel(writer,'ActiveTD Data',index = False)
pd_data.to_excel(writer,'PD Data',index = False)
writer.save()

disbursed_data_clix = disbursed_data[disbursed_data['Lender: Account Name'] == 'Clix Capital']
disbursed_data_clix = pd.merge(disbursed_data_clix, clix_dob, on = 'Loan App Reference Number', how = 'left')
disbursed_data_clix['Borrower: Date of Birth'] = disbursed_data_clix['DOB2']
disbursed_data_clix = disbursed_data_clix.drop(['DOB2'], axis=1)

td_data_clix = td_data[td_data['Lender: Account Name'] == 'Clix Capital']
pd_data_clix = pd_data[pd_data['Lender: Account Name'] == 'Clix Capital']

writer = pd.ExcelWriter(report_folder+"/"+curr_mon+"/clix_disbursed_data_"+curr_date+".xlsx")
disbursed_data_clix.to_excel(writer,'Master Data',index = False)
td_data_clix.to_excel(writer,'ActiveTD Data',index = False)
pd_data_clix.to_excel(writer,'PD Data',index = False)
writer.save()

disbursed_data_fed = disbursed_data[disbursed_data['Lender: Account Name'] == 'Federal Bank']
disbursed_data_fed = pd.merge(disbursed_data_fed, fed_map, on = 'Loan App Reference Number', how = 'left')
td_data_fed = td_data[td_data['Lender: Account Name'] == 'Federal Bank']
td_data_fed = pd.merge(td_data_fed, fed_map, on = 'Loan App Reference Number', how = 'left')
pd_data_fed = pd_data[pd_data['Lender: Account Name'] == 'Federal Bank']
pd_data_fed = pd.merge(pd_data_fed, fed_map, on = 'Loan App Reference Number', how = 'left')

writer = pd.ExcelWriter(report_folder+"/"+curr_mon+"/federal_disbursed_data_"+curr_date+".xlsx")
disbursed_data_fed.to_excel(writer,'Master Data',index = False)
td_data_fed.to_excel(writer,'ActiveTD Data',index = False)
pd_data_fed.to_excel(writer,'PD Data',index = False)
writer.save()

print("Step 4 completed successfully: Files are Ready!!")

Step 4 completed successfully: Files are Ready!!
